### Import một số thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np

### 1. Đọc dữ liệu và xử lý NaN

In [2]:
df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition_DIRTY.csv', na_values=["na", "nn", "nan","NaN", "null", "none", " "])
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
print(f"📥 Đã đọc: {df.shape[0]} hàng, {df.shape[1]} cột")
df.head()

📥 Đã đọc: 1608 hàng, 36 cột


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,DateOfResignation
0,41.0,Yes,Travel_Rarely,1091.0,Sales,2.0,2.0,Life Sciences,1.0,74.0,...,80.0,0.0,8.0,0.0,1.0,6.0,4.0,0.0,5.0,11/25/2021 10:30
1,49.0,No,Travel_Frequently,2523.0,Research & Development,8.0,1.0,Life Sciences,1.0,48.0,...,80.0,1.0,11.0,3.0,3.0,10.0,7.0,19.0,7.0,NaN
2,37.0,Yes,Travel_Rarely,1353.0,Research & Development,2.0,2.0,Other,1.0,38.0,...,80.0,0.0,7.0,3.0,3.0,0.0,0.0,0.0,0.0,4/12/2017 10:30
3,32.0,No,Travel_Frequently,1404.0,RESEARCH & DEVELOPMENT,4.0,4.0,Life Sciences,1.0,-3.0,...,80.0,0.0,8.0,3.0,3.0,NaN,7.0,3.0,0.0,NaN
4,27.0,No,Travel_Rarely,626.0,Research & Development,3.0,1.0,Medical,1.0,41.0,...,80.0,1.0,6.0,3.0,3.0,2.0,2.0,2.0,2.0,NaN


### 2. Xoá trùng lặp


In [3]:
before = df.shape[0]
df = df.drop_duplicates()
print(f"Đã xoá {before - df.shape[0]} dòng trùng lặp")

Đã xoá 138 dòng trùng lặp


### 3. Xử lý missing values


In [4]:
# Biến toàn cục để loại trừ cột không xử lý missing value
EXCLUDED_COLS = ['DateOfResignation']

# 1. Đếm giá trị NaN trước xử lý (loại trừ EXCLUDED_COLS)
missing_before = df.drop(columns=EXCLUDED_COLS, errors='ignore').isna().sum().sum()
print(f"Số giá trị còn thiếu trước xử lý (loại trừ {EXCLUDED_COLS}): {missing_before}")

# 2. Xác định cột số và cột chuỗi, trừ cột bị loại
num_cols = df.select_dtypes(include=np.number).columns.difference(EXCLUDED_COLS)
cat_cols = df.select_dtypes(include='object').columns.difference(EXCLUDED_COLS)

# 3. Cột số → điền giá trị trung bình
df[num_cols] = df[num_cols].fillna(df[num_cols].mean(numeric_only=True))

# 4. Cột chuỗi → điền giá trị phổ biến nhất (mode)
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mode(dropna=True)[0])

# 5. In thông tin sau xử lý
missing_after = df.drop(columns=EXCLUDED_COLS, errors='ignore').isna().sum().sum()
print("Đã xử lý missing values (trừ cột được loại).")
print(f"Số giá trị còn thiếu sau xử lý (loại trừ {EXCLUDED_COLS}): {missing_after}")

df.head()


Số giá trị còn thiếu trước xử lý (loại trừ ['DateOfResignation']): 2455
Đã xử lý missing values (trừ cột được loại).
Số giá trị còn thiếu sau xử lý (loại trừ ['DateOfResignation']): 0


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,DateOfResignation
0,41.0,Yes,Travel_Rarely,1091.0,Sales,2.0,2.0,Life Sciences,1.0,74.0,...,80.0,0.0,8.0,0.0,1.0,6.000000,4.0,0.0,5.0,11/25/2021 10:30
1,49.0,No,Travel_Frequently,2523.0,Research & Development,8.0,1.0,Life Sciences,1.0,48.0,...,80.0,1.0,11.0,3.0,3.0,10.000000,7.0,19.0,7.0,NaN
2,37.0,Yes,Travel_Rarely,1353.0,Research & Development,2.0,2.0,Other,1.0,38.0,...,80.0,0.0,7.0,3.0,3.0,0.000000,0.0,0.0,0.0,4/12/2017 10:30
3,32.0,No,Travel_Frequently,1404.0,RESEARCH & DEVELOPMENT,4.0,4.0,Life Sciences,1.0,-3.0,...,80.0,0.0,8.0,3.0,3.0,7.301013,7.0,3.0,0.0,NaN
4,27.0,No,Travel_Rarely,626.0,Research & Development,3.0,1.0,Medical,1.0,41.0,...,80.0,1.0,6.0,3.0,3.0,2.000000,2.0,2.0,2.0,NaN


### 4. Chuẩn hoá chuỗi: strip + title

In [5]:
# Hàm chuẩn hoá: loại bỏ khoảng trắng, chuẩn hóa chữ viết thường
def clean_text(text):
    if pd.isna(text): return text
    return str(text).strip().lower()

# Áp dụng trước: chuẩn hoá sơ bộ tất cả chuỗi
for col in cat_cols:
    df[col] = df[col].apply(clean_text)

# Với các cột dễ bị lỗi chính tả → chỉ giữ top phổ biến
for col in ['Department', 'JobRole', 'BusinessTravel']:
    top_values = df[col].value_counts().index[:3]  # Top 3 giá trị đúng thường gặp nhất
    df[col] = df[col].apply(lambda x: x if x in top_values else np.nan)
    df[col] = df[col].fillna(df[col].mode(dropna=True)[0])

# Optional: viết hoa chữ cái đầu sau khi làm sạch
for col in cat_cols:
    df[col] = df[col].apply(lambda x: str(x).title() if pd.notna(x) else x)

# In 5 giá trị đầu mỗi cột chuỗi
print("🔤 Đã chuẩn hoá + lọc lỗi chính tả bằng tần suất:")
for col in cat_cols:
    print(f"\n Cột `{col}` sau xử lý:")
    print(df[col].dropna().unique()[:5])

🔤 Đã chuẩn hoá + lọc lỗi chính tả bằng tần suất:

 Cột `Attrition` sau xử lý:
['Yes' 'No']

 Cột `BusinessTravel` sau xử lý:
['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']

 Cột `Department` sau xử lý:
['Sales' 'Research & Development' 'Human Resources']

 Cột `EducationField` sau xử lý:
['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree']

 Cột `Gender` sau xử lý:
['Female' 'Male']

 Cột `JobRole` sau xử lý:
['Sales Executive' 'Laboratory Technician' 'Research Scientist']

 Cột `MaritalStatus` sau xử lý:
['Single' 'Married' 'Divorced']

 Cột `Over18` sau xử lý:
['Y']

 Cột `OverTime` sau xử lý:
['Yes' 'No']


### 5. Làm tròn tất cả cột số nếu có thể

In [6]:
rounded_cols = [] 

for col in num_cols:
    if (df[col].dropna() % 1 == 0).all():
        df[col] = df[col].astype(int)
        rounded_cols.append(col)
    else:
        # Nếu có số thập phân, thử làm tròn trước
        df[col] = df[col].round()
        # Sau khi làm tròn, nếu tất cả đều là số nguyên thì ép kiểu int
        if (df[col].dropna() % 1 == 0).all():
            df[col] = df[col].astype(int)
            rounded_cols.append(col)

if rounded_cols:
    print("Các cột số đã được làm tròn và ép về kiểu `int`:")
    for col in rounded_cols:
        print(f"  - {col}")
else:
    print("Không có cột số nào đủ điều kiện để ép về kiểu `int`.")

Các cột số đã được làm tròn và ép về kiểu `int`:
  - Age
  - DailyRate
  - DistanceFromHome
  - Education
  - EmployeeCount
  - EmployeeNumber
  - EnvironmentSatisfaction
  - HourlyRate
  - JobInvolvement
  - JobLevel
  - JobSatisfaction
  - MonthlyIncome
  - MonthlyRate
  - NumCompaniesWorked
  - PercentSalaryHike
  - PerformanceRating
  - RelationshipSatisfaction
  - StandardHours
  - StockOptionLevel
  - TotalWorkingYears
  - TrainingTimesLastYear
  - WorkLifeBalance
  - YearsAtCompany
  - YearsInCurrentRole
  - YearsSinceLastPromotion
  - YearsWithCurrManager


### 6. Clip outliers bằng IQR

In [7]:
# Hàm clip giá trị ngoại lai bằng IQR
def clip_outliers(df, cols):
    clipped_cols = []  # Danh sách cột đã được xử lý outliers

    for col in cols:
        Q1 = df[col].quantile(0.25)  # Phân vị thứ 1 (25%)
        Q3 = df[col].quantile(0.75)  # Phân vị thứ 3 (75%)
        IQR = Q3 - Q1  # Khoảng tứ phân vị

        lower = Q1 - 1.5 * IQR  # Ngưỡng dưới
        upper = Q3 + 1.5 * IQR  # Ngưỡng trên

        # Lưu lại cột nếu có ít nhất 1 giá trị bị clip
        if ((df[col] < lower) | (df[col] > upper)).any():
            clipped_cols.append(col)

        # Áp dụng clipping
        df[col] = df[col].clip(lower, upper)

    # In kết quả
    if clipped_cols:
        print("Đã xử lý outliers bằng IQR cho các cột:")
        for col in clipped_cols:
            print(f"  - {col}")
    else:
        print("Không phát hiện outliers vượt ngưỡng IQR nào cần clip.")

    return df

# Gọi hàm xử lý
df = clip_outliers(df, num_cols)

#7. Làm tròn lại nếu clip tạo số thập phân
for col in num_cols:
    df[col] = df[col].round()

converted_to_int = []
for col in num_cols:
    if (df[col].dropna() % 1 == 0).all():  # Kiểm tra xem tất cả đều là số nguyên
        df[col] = df[col].astype(int)
        converted_to_int.append(col)

if converted_to_int:
    print("Các cột đã được ép kiểu về int:", converted_to_int)
else:
    print("ℹKhông có cột nào đủ điều kiện để ép về int.")

Đã xử lý outliers bằng IQR cho các cột:
  - Age
  - DailyRate
  - DistanceFromHome
  - Education
  - EmployeeNumber
  - HourlyRate
  - MonthlyIncome
  - MonthlyRate
  - NumCompaniesWorked
  - PercentSalaryHike
  - PerformanceRating
  - StockOptionLevel
  - TotalWorkingYears
  - TrainingTimesLastYear
  - YearsAtCompany
  - YearsInCurrentRole
  - YearsSinceLastPromotion
  - YearsWithCurrManager
Các cột đã được ép kiểu về int: ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']


### Loại bỏ các dòng có giá trị âm trong bất kỳ cột số nào


In [8]:
num_cols = df.select_dtypes(include=[np.number]).columns

negative_mask = (df[num_cols] < 0).any(axis=1)

rows_with_negatives = df[negative_mask].shape[0]

negatives = {}
for col in num_cols:
    if (df[col] < 0).any():
        negatives[col] = df[df[col] < 0][col].unique()
if negatives:
    print("Các cột có chứa giá trị âm:")
    for col, vals in negatives.items():
        print(f"  - {col}: {list(vals)}")
else:
    print("Không phát hiện giá trị âm trong bất kỳ cột số nào.")
df = df[~negative_mask]
print(f"Đã loại bỏ {rows_with_negatives} dòng có giá trị âm trong cột số")


Các cột có chứa giá trị âm:
  - EmployeeNumber: [np.int64(-3), np.int64(-47), np.int64(-31), np.int64(-38), np.int64(-51), np.int64(-23), np.int64(-14)]
  - YearsAtCompany: [np.int64(-1)]
  - YearsSinceLastPromotion: [np.int64(-1)]
Đã loại bỏ 10 dòng có giá trị âm trong cột số


### 8. Lưu kết quả


In [ ]:
output_path = "./dataset/hr_employee_attrition.csv"
df.to_csv(output_path, index=False)
print(f"Đã lưu file sạch tại: {output_path}")


Đã lưu file sạch tại: ./hr_employee_attrition.csv


### 5 dòng đầu sau khi làm sạch

In [10]:
df2 = pd.read_csv(output_path)
df2.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,DateOfResignation
0,41,Yes,Travel_Rarely,1091,Sales,2,2,Life Sciences,1,74,...,80,0,8,0,1,6,4,0,5,11/25/2021 10:30
1,49,No,Travel_Frequently,2143,Research & Development,8,1,Life Sciences,1,48,...,80,1,11,3,3,10,7,5,7,NaN
2,37,Yes,Travel_Rarely,1353,Research & Development,2,2,Other,1,38,...,80,0,7,3,3,0,0,0,0,4/12/2017 10:30
3,27,No,Travel_Rarely,626,Research & Development,3,1,Medical,1,41,...,80,1,6,3,3,2,2,2,2,NaN
4,60,No,Travel_Rarely,1348,Research & Development,2,3,Medical,1,45,...,80,2,12,3,2,1,0,0,0,NaN
